In [1]:
# Install PySpark if it's not already installed
#!pip install pyspark

from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("DataCleaning") \
    .getOrCreate()

# Load the dataset
file_path = 'csv uncleaned (1).csv'

try:
    # Read the CSV file into a DataFrame
    data = spark.read.csv(file_path, header=True, inferSchema=True)
    # Show the first few rows to verify the load
    data.show()
except Exception as e:
    print(f"An error occurred: {e}")


+-------+------+-------------+------------------+----------------+--------------------+------------------+-----------+--------------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+--------------------+--------------------+---------+---------------------+---------------+--------------+-----------------+-----+---------------+----------+------------+-------------+--------------------+----------------+--------+
|  State|   Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|     LastCheckupTime|PhysicalActivities| SleepHours|        RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|DifficultyDre

In [2]:
from pyspark.sql.functions import col, isnan, when, count

# Count missing values in each column
missing_values = data.select(
    [count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in data.columns]
)
missing_values.show()


+-----+-----+-------------+------------------+----------------+---------------+------------------+----------+------------+--------------+---------+---------+---------+-------------+-------+---------------------+----------------+------------+-----------+-------------------+-----------------------+-----------------------+-----------------+-------------------------+-----------------+------------+---------------+---------+---------------------+-----------+--------------+-----------------+-----+---------------+----------+------------+-------------+-----------------+----------------+--------+
|State|  Sex|GeneralHealth|PhysicalHealthDays|MentalHealthDays|LastCheckupTime|PhysicalActivities|SleepHours|RemovedTeeth|HadHeartAttack|HadAngina|HadStroke|HadAsthma|HadSkinCancer|HadCOPD|HadDepressiveDisorder|HadKidneyDisease|HadArthritis|HadDiabetes|DeafOrHardOfHearing|BlindOrVisionDifficulty|DifficultyConcentrating|DifficultyWalking|DifficultyDressingBathing|DifficultyErrands|SmokerStatus|ECigarette

In [3]:
from pyspark.sql.functions import col, mean

# Identify numerical columns
numeric_cols = [c for c, t in data.dtypes if t in ['int', 'double']]

# Calculate mean and fill missing values
for col_name in numeric_cols:
    mean_value = data.select(mean(col(col_name))).first()[0]
    data = data.na.fill({col_name: mean_value})


In [4]:
from pyspark.sql.functions import col, count

# Get categorical columns
categorical_cols = [c for c, t in data.dtypes if t == 'string']

# Iterate over each categorical column
for col_name in categorical_cols:
    # Get the most frequent value (mode)
    mode_df = data.groupBy(col_name).agg(count(col_name).alias('count')).orderBy('count', ascending=False)
    
    # Check if the DataFrame is empty
    if mode_df.count() > 0:
        mode_value = mode_df.first()[0]
        
        # Fill missing values with mode
        data = data.na.fill({col_name: mode_value})
    else:
        print(f"No mode found for column {col_name}.")


In [5]:
data = data.dropDuplicates()


In [6]:
replacements = {
    'Yse': 'Yes',
    'sYe': 'Yes',
    'eYs': 'Yes',
    'oN': 'No',
      # Correcting to 'Very good'
    'eVry good': 'Very good',
    'Ver ygood': 'Very good',
    'Very ogod': 'Very good',
    'Veryg oog': 'Very good',
    'Very godo': 'Very good',
    'Veyr good': 'Very good',
    'Vrey good': 'Very good',

    # Correcting to 'Fair'
    'Fari': 'Fair',
    'Fiar': 'Fair',

    # Correcting to 'Excellent'
    'Ecxellent': 'Excellent',
    'Excelletn': 'Excellent',
    'Excellnet': 'Excellent',
    'Excelelnt': 'Excellent',
    'Exclelent': 'Excellent',
    'Execllent': 'Excellent',
    'xEcellent': 'Excellent',

    # Correcting to 'Good'
    'Godo': 'Good',
    'oGod': 'Good',

    # Correcting to 'Poor'
    'oPor': 'Poor',
    'Poro': 'Poor',
    'Within pasty ear': 'Within past year',
    'oN': 'No',

        # Correcting to 'Male'
    'aMle': 'Male',
    'Mael': 'Male',
    'Mlae': 'Male',

    # Correcting to 'Female'
    'eFmale': 'Female',
    'Feamle': 'Female',
    'Femael': 'Female',
    'Femlae': 'Female',
    'Fmeale': 'Female',
    #Correcting 'LastCheckupTime' column
    'Within past year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within pasty ear (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    '5 or more years ago': '5 or more years ago',
    'Within past year (antyime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Withinp ast year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (naytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12 omnths ago)': 'Within past year (anytime less than 12 months ago)',
    'Witihn past year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less tha n12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Wihtin past year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12m onths ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12 montsh ago)': 'Within past year (anytime less than 12 months ago)',
    'Within pas tyear (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Withni past year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime les sthan 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within pats year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime elss than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12 months gao)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12 month sago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime lses than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past yea r(anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past year (ayntime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytimel ess than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytmie less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than1 2 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less htan 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Wtihin past year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past eyar (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'iWthin past year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12 monhts ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 1 2months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past yera (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year( anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Withi npast year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within apst 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Withi npast year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within apst year (anytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less than 12 mnoths ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year a(nytime less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    '5 or more years ago': '5 or more years ago',
    'Within past 2 yeasr (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past year (anytime lesst han 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    '5 or mroe years ago': '5 or more years ago',
    'Within past 2 years (1 year but less than 2 year sago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past year (anytime less than 12 months ag)o': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytim eless than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past 2 eyars (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past year (anytiem less than 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past year (anytime less thna 12 months ago)': 'Within past year (anytime less than 12 months ago)',
    '5 or moer years ago': '5 or more years ago',
    'Within past 2 years ( 1year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past year (anytime less than 12 monthsa go)': 'Within past year (anytime less than 12 months ago)',
    'Within past 2 years (1 year but less thna 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5y ears (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    '5 or more years aog': '5 or more years ago',
    'Wtihin past 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year but lesst han 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years but less than 5 year sago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past year (anytime less than 21 months ago)': 'Within past year (anytime less than 12 months ago)',
    'Within past 5 years (2 yearsb ut less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year but less than 2 years aog)': 'Within past 2 years (1 year but less than 2 years ago)',
    '5 or more yearsa go': '5 or more years ago',
    '5 or more yaers ago': '5 or more years ago',
    'Within past 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year but lses than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    '5 or more eyars ago': '5 or more years ago',
    'Within pats 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 year s(2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 yaer but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 eyar but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Withni past 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years but less tha n5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    '5 or mor eyears ago': '5 or more years ago',
    'Within past 5 years ( 2years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 eyars but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 yera but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within pas t2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within psat 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within pats 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2y ears (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but elss than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years 1( year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1y ear but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    '5 ro more years ago': '5 or more years ago',
    'Within past 2 years (1 yearb ut less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but less tahn 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year btu less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but less than 2y ears ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years bu tless than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year ubt less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but less tahn 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    '5 or more year sago': '5 or more years ago',
    'Within past 2 years (1 year but less than 2 yearsa go)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but les sthan 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past5  years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2y ears but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years butl ess than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year butl ess than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    '5 or more years gao': '5 or more years ago',
    'Within pas t5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years but less than 5 eyars ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within apst 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year bu tless than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'iWthin past 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'iWthin past 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Withi npast 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years but less than5  years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within psat 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but less than 2 yeasr ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 years (1 year but less than 2 eyars ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years but lesst han 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    '5 or morey ears ago': '5 or more years ago',
    'Within past 2 years (1 year but less than2  years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Witihn past 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years but less than  5years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Withi npast 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Withinp ast 5 years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year but less than 2 yeras ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Wihtin past 2 years (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 2 yaers (1 year but less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years but less than 5 years aog)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past  5years (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 yeras (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 yaers (2 years but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 year but less than 2 years gao)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years but less than 5 yeras ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 yaers but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 2 years (1 yea rbut less than 2 years ago)': 'Within past 2 years (1 year but less than 2 years ago)',
    'Within past 5 years (2 years but less tahn 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years but elss than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years but less thna 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years btu less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 yeasr but less than 5 years ago)': 'Within past 5 years (2 years but less than 5 years ago)',
    'Within past 5 years (2 years but less than 5 years gao)': 'Within past 5 years (2 years but less than 5 years ago)',

    #for  'RemovedTeeth' column
    
    'None of them': 'None of them',
    '6 or more, but not all': '6 or more, but not all',
    '1 to 5': '1 to 5',
    '1 to5 ': '1 to 5',
    '1 t o5': '1 to 5',
    'lAl': 'All',
    '1to 5': '1 to 5',
    '1 ot 5': '1 to 5',
    'None of thme': 'None of them',
    '6o r more, but not all': '6 or more, but not all',
    '6 or more, btu not all': '6 or more, but not all',
    ' 6or more, but not all': '6 or more, but not all',
    '6 or nota ll': '6 or more, but not all',
    '1t o 5': '1 to 5',
    'None of tehm': 'None of them',
    'None oft hem': 'None of them',
    'Noen of them': 'None of them',
    '6 or mroe, but not all': '6 or more, but not all',
    '6 or more ,but not all': '6 or more, but not all',
    '6 or more, but not lal': '6 or more, but not all',
    'None o fthem': 'None of them',
    '6 or moer, but not all': '6 or more, but not all',
    '6 or more, but ont all': '6 or more, but not all',
    'Non eof them': 'None of them',
    'Noneo f them': 'None of them',
    '6 or mor,e but not all': '6 or more, but not all',
    'None fo them': 'None of them',
    'Nnoe of them': 'None of them',
    '6 or more, but nto all': '6 or more, but not all',
    '6 orm ore, but not all': '6 or more, but not all',
    'oNne of them': 'None of them',
    '6 or more, but no tall': '6 or more, but not all',
    '6 or more, butn ot all': '6 or more, but not all',
    '6 ro more, but not all': '6 or more, but not all',
    '6 o rmore, but not all': '6 or more, but not all',
    '6 or more, ubt not all': '6 or more, but not all',
    'None of htem': 'None of them',
    '6 or omre, but not all': '6 or more, but not all',
    '6 or more, bu tnot all': '6 or more, but not all',
    '6 or more,b ut not all': '6 or more, but not all',

    #for 'State ' column
    
    'Alabama': 'Alabama',
    'Albaama': 'Alabama',
    'Alaabma': 'Alabama',
    'Aalbama': 'Alabama',
    'Alabmaa': 'Alabama',
    'Alabaam': 'Alabama',
    'lAabama': 'Alabama',
    'Alaska': 'Alaska',
    'Alasak': 'Alaska',
    'Alsaka': 'Alaska',
    'lAaska': 'Alaska',
    'Alaksa': 'Alaska',
    'Aalska': 'Alaska',
    'Ariznoa': 'Arizona',
    'Arizona': 'Arizona',
    'Arziona': 'Arizona',
    'Ariozna': 'Arizona',
    'Airzona': 'Arizona',
    'Arizoan': 'Arizona',
    'rAizona': 'Arizona',
    'Arkansas': 'Arkansas',
    'Arkanass': 'Arkansas',
    'Arkanssa': 'Arkansas',
    'Arkasnas': 'Arkansas',
    'Araknsas': 'Arkansas',
    'Akransas': 'Arkansas',
    'Arknasas': 'Arkansas',
    'rAkansas': 'Arkansas',
    'California': 'California',
    'Claifornia': 'California',
    'Cailfornia': 'California',
    'Caliofrnia': 'California',
    'Calfiornia': 'California',
    'Califronia': 'California',
    'aClifornia': 'California',
    'Califorina': 'California',
    'Califonria': 'California',
    'Californai': 'California',
    'Coloardo': 'Colorado',
    'Colorado': 'Colorado',
    'Cloorado': 'Colorado',
    'Coloraod': 'Colorado',
    'Colroado': 'Colorado',
    'Colordao': 'Colorado',
    'Coolrado': 'Colorado',
    'oClorado': 'Colorado',
    'Connecticut': 'Connecticut',
    'Connetcicut': 'Connecticut',
    'Connectiuct': 'Connecticut',
    'oCnnecticut': 'Connecticut',
    'Connectciut': 'Connecticut',
    'Conencticut': 'Connecticut',
    'Connceticut': 'Connecticut',
    'Connectictu': 'Connecticut',
    'Cnonecticut': 'Connecticut',
    'Connecitcut': 'Connecticut',
    'Delaware': 'Delaware',
    'Dleaware': 'Delaware',
    'Delawaer': 'Delaware',
    'eDlaware': 'Delaware',
    'Delawrae': 'Delaware',
    'Dealware': 'Delaware',
    'Delwaare': 'Delaware',
    'Delaawre': 'Delaware',
    'Distirct of Columbia': 'District of Columbia',
    'District of Columbia': 'District of Columbia',
    'Dsitrict of Columbia': 'District of Columbia',
    'District of Coulmbia': 'District of Columbia',
    'District o fColumbia': 'District of Columbia',
    'iDstrict of Columbia': 'District of Columbia',
    'District of Colmubia': 'District of Columbia',
    'Distrcit of Columbia': 'District of Columbia',
    'Distritc of Columbia': 'District of Columbia',
    'Distric tof Columbia': 'District of Columbia',
    'District of Columbai': 'District of Columbia',
    'District of Cloumbia': 'District of Columbia',
    'Districto f Columbia': 'District of Columbia',
    'District ofC olumbia': 'District of Columbia',
    'District of Columiba': 'District of Columbia',
    'District of Colubmia': 'District of Columbia',
    'Ditsrict of Columbia': 'District of Columbia',
    'District fo Columbia': 'District of Columbia',
    'District of oClumbia': 'District of Columbia',
    'Disrtict of Columbia': 'District of Columbia',
    'Florida': 'Florida',
    'lForida': 'Florida',
    'Folrida': 'Florida',
    'Flordia': 'Florida',
    'Floriad': 'Florida',
    'Flroida': 'Florida',
    'Floirda': 'Florida',
    'Georgia': 'Georgia',
    'Gerogia': 'Georgia',
    'Goergia': 'Georgia',
    'eGorgia': 'Georgia',
    'Georiga': 'Georgia',
    'Geogria': 'Georgia',
    'Georgai': 'Georgia',
    'Hawaii': 'Hawaii',
    'aHwaii': 'Hawaii',
    'Haawii': 'Hawaii',
    'Hwaaii': 'Hawaii',
    'Hawiai': 'Hawaii',
    'Idaho': 'Idaho',
    'Idaoh': 'Idaho',
    'dIaho': 'Idaho',
    'Iadho': 'Idaho',
    'Idhao': 'Idaho',
    'Illinois': 'Illinois',
    'Illinios': 'Illinois',
    'Ililnois': 'Illinois',
    'Illniois': 'Illinois',
    'Illinosi': 'Illinois',
    'Illionis': 'Illinois',
    'lIlinois': 'Illinois',
    'Indiana': 'Indiana',
    'nIdiana': 'Indiana',
    'Indinaa': 'Indiana',
    'Idniana': 'Indiana',
    'Inidana': 'Indiana',
    'Indiaan': 'Indiana',
    'Indaina': 'Indiana',
    'Iowa': 'Iowa',
    'Iwoa': 'Iowa',
    'Ioaw': 'Iowa',
    'oIwa': 'Iowa',
    'Kansas': 'Kansas',
    'Kanssa': 'Kansas',
    'Knasas': 'Kansas',
    'Kasnas': 'Kansas',
    'aKnsas': 'Kansas',
    'Kanass': 'Kansas',
    'Kentucky': 'Kentucky',
    'Kentukcy': 'Kentucky',
    'Kentcuky': 'Kentucky',
    'Kentucyk': 'Kentucky',
    'Ketnucky': 'Kentucky',
    'eKntucky': 'Kentucky',
    'Knetucky': 'Kentucky',
    'Kenutcky': 'Kentucky',
    'Louisiana': 'Louisiana',
    'Louisaina': 'Louisiana',
    'Louisiaan': 'Louisiana',
    'Louiisana': 'Louisiana',
    'Luoisiana': 'Louisiana',
    'oLuisiana': 'Louisiana',
    'Louisinaa': 'Louisiana',
    'Loiusiana': 'Louisiana',
    'Lousiiana': 'Louisiana',
    'Maine': 'Maine',
    'Miane': 'Maine',
    'Manie': 'Maine',
    'aMine': 'Maine',
    'Maien': 'Maine',
    'Marlyand': 'Maryland',
    'Maryland': 'Maryland',
    'Marylnad': 'Maryland',
    'aMryland': 'Maryland',
    'Mayrland': 'Maryland',
    'Maryladn': 'Maryland',
    'Mrayland': 'Maryland',
    'Maryalnd': 'Maryland',
    'Massachusetts': 'Massachusetts',
    'Massachuestts': 'Massachusetts',
    'Msasachusetts': 'Massachusetts',
    'Massacuhsetts': 'Massachusetts',
    'Massachusetst': 'Massachusetts',
    'Massachustets': 'Massachusetts',
    'Massachsuetts': 'Massachusetts',
    'aMssachusetts': 'Massachusetts',
    'Masaschusetts': 'Massachusetts',
    'Massahcusetts': 'Massachusetts',
    'Masscahusetts': 'Massachusetts',
    'Michigan': 'Michigan',
    'Michgian': 'Michigan',
    'Michigna': 'Michigan',
    'Michiagn': 'Michigan',
    'Mcihigan': 'Michigan',
    'Mihcigan': 'Michigan',
    'Micihgan': 'Michigan',
    'iMchigan': 'Michigan',
    'Minnesota': 'Minnesota',
    'Mninesota': 'Minnesota',
    'Minnseota': 'Minnesota',
    'Minnestoa': 'Minnesota',
    'Minnesoat': 'Minnesota',
    'Minensota': 'Minnesota',
    'iMnnesota': 'Minnesota',
    'Minneosta': 'Minnesota',
    'Mississippi': 'Mississippi',
    'Msisissippi': 'Mississippi',
    'Misssisippi': 'Mississippi',
    'iMssissippi': 'Mississippi',
    'Mississipip': 'Mississippi',
    'Misisssippi': 'Mississippi',
    'Missisisppi': 'Mississippi',
    'Mississpipi': 'Mississippi',
    'Missouri': 'Missouri',
    'Misosuri': 'Missouri',
    'iMssouri': 'Missouri',
    'Missouir': 'Missouri',
    'Missuori': 'Missouri',
    'Missorui': 'Missouri',
    'Msisouri': 'Missouri',
    'Montana': 'Montana',
    'Montaan': 'Montana',
    'Montnaa': 'Montana',
    'Motnana': 'Montana',
    'Monatna': 'Montana',
    'oMntana': 'Montana',
    'Mnotana': 'Montana',
    'Nebraska': 'Nebraska',
    'Nberaska': 'Nebraska',
    'Nerbaska': 'Nebraska',
    'Nebrsaka': 'Nebraska',
    'Nebarska': 'Nebraska',
    'Nebrasak': 'Nebraska',
    'eNbraska': 'Nebraska',
    'Nebraksa': 'Nebraska',
    'Nevada': 'Nevada',
    'Nevdaa': 'Nevada',
    'Nveada': 'Nevada',
    'Nevaad': 'Nevada',
    'eNvada': 'Nevada',
    'Neavda': 'Nevada',
    'New Hampshire': 'New Hampshire',
    'Ne wHampshire': 'New Hampshire',
    'eNw Hampshire': 'New Hampshire',
    'New Hampshrie': 'New Hampshire',
    'New Hampsihre': 'New Hampshire',
    'Nwe Hampshire': 'New Hampshire',
    'New aHmpshire': 'New Hampshire',
    'New Hamphsire': 'New Hampshire',
    'New Hamsphire': 'New Hampshire',
    'New Hapmshire': 'New Hampshire',
    'New Hampshier': 'New Hampshire',
    'NewH ampshire': 'New Hampshire',
    'New Hmapshire': 'New Hampshire',
    'New Jersey': 'New Jersey',
    'Nwe Jersey': 'New Jersey',
    'New Jesrey': 'New Jersey',
    'Ne wJersey': 'New Jersey',
    'NewJ ersey': 'New Jersey',
    'eNw Jersey': 'New Jersey',
    'New Jeresy': 'New Jersey',
    'New Jersye': 'New Jersey',
    'New Jresey': 'New Jersey',
    'New eJrsey': 'New Jersey',
    'New Mexico': 'New Mexico',
    'New Mexioc': 'New Mexico',
    'Ne wMexico': 'New Mexico',
    'New Mxeico': 'New Mexico',
    'New Meixco': 'New Mexico',
    'eNw Mexico': 'New Mexico',
    'New eMxico': 'New Mexico',
    'Nwe Mexico': 'New Mexico',
    'NewM exico': 'New Mexico',
    'New Mexcio': 'New Mexico',
    'New Yokr': 'New York',
    'New York': 'New York',
    'New Yrok': 'New York',
    'NewY ork': 'New York',
    'Ne wYork': 'New York',
    'eNw York': 'New York',
    'Nwe York': 'New York',
    'New oYrk': 'New York',
    'North Carolina': 'North Carolina',
    'North Caorlina': 'North Carolina',
    'Notrh Carolina': 'North Carolina',
    'NorthC arolina': 'North Carolina',
    'North Caroilna': 'North Carolina',
    'North aCrolina': 'North Carolina',
    'North Carolnia': 'North Carolina',
    'Norht Carolina': 'North Carolina',
    'Nort hCarolina': 'North Carolina',
    'North Carloina': 'North Carolina',
    'North Craolina': 'North Carolina',
    'Nroth Carolina': 'North Carolina',
    'oNrth Carolina': 'North Carolina',
    'North Carolian': 'North Carolina',
    'North Dakota': 'North Dakota',
    'North aDkota': 'North Dakota',
    'oNrth Dakota': 'North Dakota',
    'Nort hDakota': 'North Dakota',
    'North Daokta': 'North Dakota',
    'North Dkaota': 'North Dakota',
    'Nroth Dakota': 'North Dakota',
    'North Daktoa': 'North Dakota',
    'North Dakoat': 'North Dakota',
    'NorthD akota': 'North Dakota',
    'Notrh Dakota': 'North Dakota',
    'Norht Dakota': 'North Dakota',
    'Ohio': 'Ohio',
    'hOio': 'Ohio',
    'Oiho': 'Ohio',
    'Ohoi': 'Ohio',
    'Oklahoma': 'Oklahoma',
    'Olkahoma': 'Oklahoma',
    'kOlahoma': 'Oklahoma',
    'Oklahmoa': 'Oklahoma',
    'Okalhoma': 'Oklahoma',
    'Oklahoam': 'Oklahoma',
    'Oklaohma': 'Oklahoma',
    'Oklhaoma': 'Oklahoma',
    'Oregon': 'Oregon',
    'Oergon': 'Oregon',
    'Orgeon': 'Oregon',
    'rOegon': 'Oregon',
    'Oregno': 'Oregon',
    'Oreogn': 'Oregon',
    'Pennsylvania': 'Pennsylvania',
    'ePnnsylvania': 'Pennsylvania',
    'Pennsyvlania': 'Pennsylvania',
    'Pennsylvanai': 'Pennsylvania',
    'Pennyslvania': 'Pennsylvania',
    'Pennsylvaina': 'Pennsylvania',
    'Pennsylvnaia': 'Pennsylvania',
    'Pennslyvania': 'Pennsylvania',
    'Pensnylvania': 'Pennsylvania',
    'Pennsylavnia': 'Pennsylvania',
    'Pnensylvania': 'Pennsylvania',
    'Rhode Island': 'Rhode Island',
    'RhodeI sland': 'Rhode Island',
    'Rhdoe Island': 'Rhode Island',
    'Rhode sIland': 'Rhode Island',
    'Rohde Island': 'Rhode Island',
    'Rhode Ilsand': 'Rhode Island',
    'Rhode Isalnd': 'Rhode Island',
    'Rhod eIsland': 'Rhode Island',
    'hRode Island': 'Rhode Island',
    'Rhoed Island': 'Rhode Island',
    'Rhode Islnad': 'Rhode Island',
    'Rhode Isladn': 'Rhode Island',
    'South Carolina': 'South Carolina',
    'South aCrolina': 'South Carolina',
    'South Carolnia': 'South Carolina',
    'Suoth Carolina': 'South Carolina',
    'South Carloina': 'South Carolina',
    'SouthC arolina': 'South Carolina',
    'South Caroilna': 'South Carolina',
    'Sout hCarolina': 'South Carolina',
    'South Caorlina': 'South Carolina',
    'South Craolina': 'South Carolina',
    'South Carolian': 'South Carolina',
    'Sotuh Carolina': 'South Carolina',
    'oSuth Carolina': 'South Carolina',
    'Souht Carolina': 'South Carolina',
    'South Dakota': 'South Dakota',
    'South Daokta': 'South Dakota',
    'South Dkaota': 'South Dakota',
    'Sotuh Dakota': 'South Dakota',
    'Suoth Dakota': 'South Dakota',
    'South Dakoat': 'South Dakota',
    'SouthD akota': 'South Dakota',
    'South Daktoa': 'South Dakota',
    'Souht Dakota': 'South Dakota',
    'oSuth Dakota': 'South Dakota',
    'Sout hDakota': 'South Dakota',
    'South aDkota': 'South Dakota',
    'Tennessee': 'Tennessee',
    'Tennesese': 'Tennessee',
    'Tenenssee': 'Tennessee',
    'Tennsesee': 'Tennessee',
    'Tnenessee': 'Tennessee',
    'eTnnessee': 'Tennessee',
    'Texas': 'Texas',
    'Texsa': 'Texas',
    'eTxas': 'Texas',
    'Teaxs': 'Texas',
    'Txeas': 'Texas',
    'Utah': 'Utah',
    'Utha': 'Utah',
    'tUah': 'Utah',
    'Uath': 'Utah',
    'Vermont': 'Vermont',
    'Vremont': 'Vermont',
    'eVrmont': 'Vermont',
    'Vermnot': 'Vermont',
    'Vemront': 'Vermont',
    'Vermotn': 'Vermont',
    'Veromnt': 'Vermont',
    'Virginia': 'Virginia',
    'Virgniia': 'Virginia',
    'Vriginia': 'Virginia',
    'Virginai': 'Virginia',
    'Virgiina': 'Virginia',
    'Virignia': 'Virginia',
    'Vigrinia': 'Virginia',
    'iVrginia': 'Virginia',
    'Washington': 'Washington',
    'Washintgon': 'Washington',
    'Washnigton': 'Washington',
    'Wahsington': 'Washington',
    'Washingotn': 'Washington',
    'aWshington': 'Washington',
    'Washingtno': 'Washington',
    'Washignton': 'Washington',
    'Wasihngton': 'Washington',
    'Wsahington': 'Washington',
    'West Virginia': 'West Virginia',
    'West iVrginia': 'West Virginia',
    'West Vigrinia': 'West Virginia',
    'West Virignia': 'West Virginia',
    'Wets Virginia': 'West Virginia',
    'West Vriginia': 'West Virginia',
    'West Virgiina': 'West Virginia',
    'Wset Virginia': 'West Virginia',
    'West Virgniia': 'West Virginia',
    'eWst Virginia': 'West Virginia',
    'WestV irginia': 'West Virginia',
    'West Virginai': 'West Virginia',
    'Wes tVirginia': 'West Virginia',
    'Wisconsin': 'Wisconsin',
    'Wiscnosin': 'Wisconsin',
    'Wiscosnin': 'Wisconsin',
    'Wisconsni': 'Wisconsin',
    'Wisocnsin': 'Wisconsin',
    'Wisconisn': 'Wisconsin',
    'Wicsonsin': 'Wisconsin',
    'iWsconsin': 'Wisconsin',
    'Wsiconsin': 'Wisconsin',
    'Wyoming': 'Wyoming',
    'Wyoimng': 'Wyoming',
    'Woyming': 'Wyoming',
    'Wyomnig': 'Wyoming',
    'yWoming': 'Wyoming',
    'Wyomign': 'Wyoming',
    'Wymoing': 'Wyoming',
    'Guam': 'Guam',
    'Guma': 'Guam',
    'Gaum': 'Guam',
    'uGam': 'Guam',
    'Puerto Rico': 'Puerto Rico',
    'Puetro Rico': 'Puerto Rico',
    'Pureto Rico': 'Puerto Rico',
    'PuertoR ico': 'Puerto Rico',
    'Puerto iRco': 'Puerto Rico',
    'Peurto Rico': 'Puerto Rico',
    'Puerto Rcio': 'Puerto Rico',
    'uPerto Rico': 'Puerto Rico',
    'Puerot Rico': 'Puerto Rico',
    'Puerto Rioc': 'Puerto Rico',
    'Puert oRico': 'Puerto Rico',
    'Virgin Islands': 'Virgin Islands',
    'Vrigin Islands': 'Virgin Islands',
    'Virgni Islands': 'Virgin Islands',
    'Virgin Isladns': 'Virgin Islands',
    'Virgin Ilsands': 'Virgin Islands',
    'Virgin Islnads': 'Virgin Islands',
    'Virgin Islansd': 'Virgin Islands',
    'Vigrin Islands': 'Virgin Islands',
    'VirginI slands': 'Virgin Islands',
    'Virign Islands': 'Virgin Islands',
    'Virgin sIlands': 'Virgin Islands',
    'iVrgin Islands': 'Virgin Islands',
    'Virgin Isalnds': 'Virgin Islands',
    'Virgi nIslands': 'Virgin Islands',
    
    #for 'SmokerStatus ' column
    'Former smoekr': 'Former smoker',
    'Current smoker - now smokes every day': 'Current smoker - now smokes every day',
    'Former smoker': 'Former smoker',
    'Never smoked': 'Never smoked',
    'Former somker': 'Former smoker',
    'Fomrer smoker': 'Former smoker',
    'Neevr smoked': 'Never smoked',
    'Formre smoker': 'Former smoker',
    'Never somked': 'Never smoked',
    'Nveer smoked': 'Never smoked',
    'Current smoker - now smokes some days': 'Current smoker - now smokes some days',
    'Former smkoer': 'Former smoker',
    'Former smokre': 'Former smoker',
    'Former msoker': 'Former smoker',
    'Never smoekd': 'Never smoked',
    'Neve rsmoked': 'Never smoked',
    'Current smoker  -now smokes every day': 'Current smoker - now smokes every day',
    'Currents moker - now smokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now somkes every day': 'Current smoker - now smokes every day',
    'eNver smoked': 'Never smoked',
    'oFrmer smoker': 'Former smoker',
    'Nevre smoked': 'Never smoked',
    'Current smokre - now smokes every day': 'Current smoker - now smokes every day',
    'Formers moker': 'Former smoker',
    'Current msoker - now smokes every day': 'Current smoker - now smokes every day',
    'Current smoker-  now smokes every day': 'Current smoker - now smokes every day',
    'Never smokde': 'Never smoked',
    'Never msoked': 'Never smoked',
    'Nevers moked': 'Never smoked',
    'Current somker - now smokes some days': 'Current smoker - now smokes some days',
    'Forme rsmoker': 'Former smoker',
    'Current smoker - nows mokes some days': 'Current smoker - now smokes some days',
    'Fromer smoker': 'Former smoker',
    'Currents moker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - no wsmokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes soem days': 'Current smoker - now smokes some days',
    'Foremr smoker': 'Former smoker',
    'Current smoker - now smokese very day': 'Current smoker - now smokes every day',
    'Current smoker - now msokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokes everyd ay': 'Current smoker - now smokes every day',
    'Current smoker - now smokes some adys': 'Current smoker - now smokes some days',
    'Current smoker - now smokes somed ays': 'Current smoker - now smokes some days',
    'Curren tsmoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokes every ady': 'Current smoker - now smokes every day',
    'Current smoker - onw smokes some days': 'Current smoker - now smokes some days',
    'Curretn smoker - now smokes every day': 'Current smoker - now smokes every day',
    'Current smoekr - now smokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes ever yday': 'Current smoker - now smokes every day',
    'Current smoker - now smkoes every day': 'Current smoker - now smokes every day',
    'Currnet smoker - now smokes every day': 'Current smoker - now smokes every day',
    'Current somker - now smokes every day': 'Current smoker - now smokes every day',
    'Curren tsmoker - now smokes every day': 'Current smoker - now smokes every day',
    'Curernt smoker - now smokes every day': 'Current smoker - now smokes every day',
    'uCrrent smoker - now smokes every day': 'Current smoker - now smokes every day',
    'Crurent smoker - now smokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now smoke severy day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes some dasy': 'Current smoker - now smokes some days',
    'Curretn smoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoke r- now smokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes eevry day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes evrey day': 'Current smoker - now smokes every day',
    'Crurent smoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now somkes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokes some dyas': 'Current smoker - now smokes some days',
    'Current smoker - nwo smokes some days': 'Current smoker - now smokes some days',
    'Current smokre - now smokes some days': 'Current smoker - now smokes some days',
    'Current smkoer - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smkoes some days': 'Current smoker - now smokes some days',
    'Current smoker - now msokes every day': 'Current smoker - now smokes every day',
    'Current smoker - nwo smokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes eveyr day': 'Current smoker - now smokes every day',
    'Current smoker - now smoeks every day': 'Current smoker - now smokes every day',
    'Current smoekr - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokes veery day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes every dya': 'Current smoker - now smokes every day',
    'Current smoker - now smoke ssome days': 'Current smoker - now smokes some days',
    'Current msoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - no wsmokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokse some days': 'Current smoker - now smokes some days',
    'Current smoker - onw smokes every day': 'Current smoker - now smokes every day',
    'Currnet smoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokess ome days': 'Current smoker - now smokes some days',
    'Current smoker -n ow smokes every day': 'Current smoker - now smokes every day',
    'Current smoker - now smokes smoe days': 'Current smoker - now smokes some days',
    'Current smoker -n ow smokes some days': 'Current smoker - now smokes some days',
    'Current smkoer - now smokes every day': 'Current smoker - now smokes every day',
    'uCrrent smoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokse every day': 'Current smoker - now smokes every day',
    'Current smoker  -now smokes some days': 'Current smoker - now smokes some days',
    'Current smoke r- now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smokes osme days': 'Current smoker - now smokes some days',
    'Current smoker - now smokes som edays': 'Current smoker - now smokes some days',
    'Curernt smoker - now smokes some days': 'Current smoker - now smokes some days',
    'Current smoker - now smoeks some days': 'Current smoker - now smokes some days',

    #for 'ECigaretteUsage' column
    
    'Never used e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cgiarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Use them some days': 'Use them some days',
    'Not at all (right now)': 'Not at all (right now)',
    'Never used -ecigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never uesd e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigaretets in my entire life': 'Never used e-cigarettes in my entire life',
    'Never usde e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Nevre used e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used ec-igarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettesi n my entire life': 'Never used e-cigarettes in my entire life',
    'Nveer used e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my enitre life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettse in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes i nmy entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my entire ilfe': 'Never used e-cigarettes in my entire life',
    'Not at all (rightn ow)': 'Not at all (right now)',
    'Never used e-cigarettes inm y entire life': 'Never used e-cigarettes in my entire life',
    'Use them everyd ay': 'Use them every day',
    'Never used e-ciagrettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never sued e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in mye ntire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my entir elife': 'Never used e-cigarettes in my entire life',
    'Not at all( right now)': 'Not at all (right now)',
    'Neve rused e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Neevr used e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Neveru sed e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my entirel ife': 'Never used e-cigarettes in my entire life',
    'oNt at all (right now)': 'Not at all (right now)',
    'Not at all (rgiht now)': 'Not at all (right now)',
    'Never used e-cigartetes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes ni my entire life': 'Never used e-cigarettes in my entire life',
    'Not ta all (right now)': 'Not at all (right now)',
    'Not at all (right onw)': 'Not at all (right now)',
    'Never used e-cigarette sin my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my entire lfie': 'Never used e-cigarettes in my entire life',
    'Not ata ll (right now)': 'Not at all (right now)',
    'Never used e-cigarettes in ym entire life': 'Never used e-cigarettes in my entire life',
    'Use htem every day': 'Use them every day',
    'Never usede -cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my entire lief': 'Never used e-cigarettes in my entire life',
    'Never used e-cigarettes in my entier life': 'Never used e-cigarettes in my entire life',
    'No tat all (right now)': 'Not at all (right now)',
    'Use thme every day': 'Use them every day',
    'Never used e-cigarettes in my etnire life': 'Never used e-cigarettes in my entire life',
    'Use them somed ays': 'Use them some days',
    'Never use de-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Never used e-cigraettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Uset hem some days': 'Use them some days',
    'Not at all (rigth now)': 'Not at all (right now)',
    'Never used e-icgarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'Ues them some days': 'Use them some days',
    'Never used e-cigarettes in my entrie life': 'Never used e-cigarettes in my entire life',
    'Us ethem some days': 'Use them some days',
    'Use them some dasy': 'Use them some days',
    'Nto at all (right now)': 'Not at all (right now)',
    'eNver used e-cigarettes in my entire life': 'Never used e-cigarettes in my entire life',
    'sUe them some days': 'Use them some days',
    'Not at all (right nwo)': 'Not at all (right now)',
    'Not at all (irght now)': 'Not at all (right now)',
    'Use them veery day': 'Use them every day',
    'Not at al l(right now)': 'Not at all (right now)',
    'Not at lal (right now)': 'Not at all (right now)',
    'Not at all (right no)w': 'Not at all (right now)',
    'Never used e-cigarettes in my netire life': 'Never used e-cigarettes in my entire life',
    'Use them every dya': 'Use them every day',
    'Never used e-cigarettes in m yentire life': 'Never used e-cigarettes in my entire life',
    'Not at all r(ight now)': 'Not at all (right now)',
    'Not at all (righ tnow)': 'Not at all (right now)',
    'Uset hem every day': 'Use them every day',
    'Use them som edays': 'Use them some days',
    'Use thems ome days': 'Use them some days',
    'Nota t all (right now)': 'Not at all (right now)',
    'Not a tall (right now)': 'Not at all (right now)',
    'Use tehm every day': 'Use them every day',
    'Use them evrey day': 'Use them every day',
    'Us ethem every day': 'Use them every day',
    'Ues them every day': 'Use them every day',
    'Use tehm some days': 'Use them some days',
    'Use the msome days': 'Use them some days',
    'Use them smoe days': 'Use them some days',
    'Use them soem days': 'Use them some days',
    'Use them some adys': 'Use them some days',
    'Use them eveyr day': 'Use them every day',
    'Use the mevery day': 'Use them every day',
    'Use them every ady': 'Use them every day',
    'Use them osme days': 'Use them some days',
    'Use them eevry day': 'Use them every day',
    'Use htem some days': 'Use them some days',
    'Use theme very day': 'Use them every day',
    'Use thme some days': 'Use them some days',
    'sUe them every day': 'Use them every day',
    'Use them some dyas': 'Use them some days',
    'Use them ever yday': 'Use them every day',

    #for 'HadDiabetes' column
    
    'No': 'No',
    'Yes': 'Yes',
    'oN': 'No',
    'Yes, but only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yse': 'Yes',
    'No, pre-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'eYs': 'Yes',
    'Yes, but only during prengancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during preganncy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or bordreline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but onlyd uring pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but noly during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or borderlien diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnancy( female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diaebtes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pr-ediabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, rpe-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnancy (feamle)': 'Yes, but only during pregnancy (female)',
    'No, pre-daibetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetse or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No ,pre-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or bordelrine diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only druing pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabeets or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during rpegnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabeteso r borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or bodrerline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabete sor borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or borderline diaebtes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-idabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes orb orderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes o rborderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only duringp regnancy (female)': 'Yes, but only during pregnancy (female)',
    'N,o pre-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or borderlin ediabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabtees or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or borderlined iabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or borderline idabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnanc y(female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during pregnancy (efmale)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or borderline dibaetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-dibaetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnancy (femlae)': 'Yes, but only during pregnancy (female)',
    'Yes, but only durin gpregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or boredrline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnnacy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or borderline diabetse': 'No, pre-diabetes or borderline diabetes',
    'Yse, but only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes,b ut only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pred-iabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or borderline diabtees': 'No, pre-diabetes or borderline diabetes',
    'Yes, bu tonly during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or borderline daibetes': 'No, pre-diabetes or borderline diabetes',
    'No, pre-diabetes or borderilne diabetes': 'No, pre-diabetes or borderline diabetes',
    'eYs, but only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or borderlnie diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only durign pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or obrderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnacny (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during pergnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only duirng pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes or borderline diabeets': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only udring pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during pregnanyc (female)': 'Yes, but only during pregnancy (female)',
    'No, per-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during prgenancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during pregnancy (femal)e': 'Yes, but only during pregnancy (female)',
    'Yes, but olny during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, buto nly during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No,p re-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but onyl during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'oN, pre-diabetes or borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Ye,s but only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but onl yduring pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'No, pre-diabetes ro borderline diabetes': 'No, pre-diabetes or borderline diabetes',
    'Yes, but only during pregnancy (femael)': 'Yes, but only during pregnancy (female)',
    'Yes, btu only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes ,but only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only durnig pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, ubt only during pregnancy (female)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during pregnancy (fmeale)': 'Yes, but only during pregnancy (female)',
    'Yes, but only during pregnancy f(emale)': 'Yes, but only during pregnancy (female)',
    #for 'RaceEthnicityCategory' column
    
    'White only, Non-Hispanic': 'White only, Non-Hispanic',
    'Black only, Non-Hispanic': 'Black only, Non-Hispanic',
    'White noly, Non-Hispanic': 'White only, Non-Hispanic',
    'White onl,y Non-Hispanic': 'White only, Non-Hispanic',
    'White only, Non-Hispanci': 'White only, Non-Hispanic',
    'Black only, oNn-Hispanic': 'Black only, Non-Hispanic',
    'Mulitracial, Non-Hispanic': 'Multiracial, Non-Hispanic',
    'Other race only, Non-Hispanic': 'Other race only, Non-Hispanic',
    'White only, Non-Hispainc': 'White only, Non-Hispanic',
    'Hispanic': 'Hispanic',
    'Black only,N on-Hispanic': 'Black only, Non-Hispanic',
    'Other race only, No-nHispanic': 'Other race only, Non-Hispanic',
    'Other race only,N on-Hispanic': 'Other race only, Non-Hispanic',
    'Black only, Non-iHspanic': 'Black only, Non-Hispanic',
    'Black only, Non-Hipsanic': 'Black only, Non-Hispanic',
    'Black only, Non-Hispnaic': 'Black only, Non-Hispanic',
    'White only, Nno-Hispanic': 'White only, Non-Hispanic',
    'Black only, Non-Hispanci': 'Black only, Non-Hispanic',
    'Black only ,Non-Hispanic': 'Black only, Non-Hispanic',
    'Other race only ,Non-Hispanic': 'Other race only, Non-Hispanic',
    'Black noly, Non-Hispanic': 'Black only, Non-Hispanic',
    'White only, Non-Hisapnic': 'White only, Non-Hispanic',
    'Black only, Non-Hisapnic': 'Black only, Non-Hispanic',
    'lBack only, Non-Hispanic': 'Black only, Non-Hispanic',
    'Other rcae only, Non-Hispanic': 'Other race only, Non-Hispanic',
    'Multiracial, Non-Hispainc': 'Multiracial, Non-Hispanic',
    'Multiracia,l Non-Hispanic': 'Multiracial, Non-Hispanic',
    'Other arce only, Non-Hispanic': 'Other race only, Non-Hispanic',
    'Hisapnic': 'Hispanic',
    'Hsipanic': 'Hispanic',
    'uMltiracial, Non-Hispanic': 'Multiracial, Non-Hispanic',
    'Mutliracial, Non-Hispanic': 'Multiracial, Non-Hispanic',
    'Other race noly, Non-Hispanic': 'Other race only, Non-Hispanic',
    'Othe rrace only, Non-Hispanic': 'Other race only, Non-Hispanic',
    'Othre race only, Non-Hispanic': 'Other race only, Non-Hispanic',
    'Other race only, Non-Hisapnic': 'Other race only, Non-Hispanic',
    'Other race only, Non-Hispnaic': 'Other race only, Non-Hispanic',
    'Other race only, Non-Hispanci': 'Other race only, Non-Hispanic',
    'Other race only, Non-Hispainc': 'Other race only, Non-Hispanic',
    'Multiracial, Non-Hisapnic': 'Multiracial, Non-Hispanic',
    'Multiracial, Non-Hispnaic': 'Multiracial, Non-Hispanic',
    'Multiracial, NonH-ispanic': 'Multiracial, Non-Hispanic',
    'Multiracial, No-nHispanic': 'Multiracial, Non-Hispanic',
    'Multiracial, Non-Hispanci': 'Multiracial, Non-Hispanic',
    'White onyl, Non-Hispanic': 'White only, Non-Hispanic',
    'Blac konly, Non-Hispanic': 'Black only, Non-Hispanic',
    'Blakc only, Non-Hispanic': 'Black only, Non-Hispanic',
    'Multiracial, Nno-Hispanic': 'Multiracial, Non-Hispanic',
    'Mlutiracial, Non-Hispanic': 'Multiracial, Non-Hispanic',
    'Multiracial, Nno-Hispanic': 'Multiracial, Non-Hispanic',
    'Othe rrace only, Non-Hispanic': 'Other race only, Non-Hispanic',
    'Other race only, oNn-Hispanic': 'Other race only, Non-Hispanic',

    #for 'AgeCategory' column
    
    'Age 18 to 24': 'Age 18 to 24',
    'Age 25 to 29': 'Age 25 to 29',
    'Age 30 to 34': 'Age 30 to 34',
    'Age 35 to 39': 'Age 35 to 39',
    'Age 40 to 44': 'Age 40 to 44',
    'Age 45 to 49': 'Age 45 to 49',
    'Age 50 to 54': 'Age 50 to 54',
    'Age 55 to 59': 'Age 55 to 59',
    'Age 60 to 64': 'Age 60 to 64',
    'Age 65 to 69': 'Age 65 to 69',
    'Age 70 to 74': 'Age 70 to 74',
    'Age 75 to 79': 'Age 75 to 79',
    'Age 80 or older': 'Age 80 or older',
    'Age 18 ot 24': 'Age 18 to 24',
    'Age 25 ot 29': 'Age 25 to 29',
    'Age 30 to3 4': 'Age 30 to 34',
    'Age 35 t o39': 'Age 35 to 39',
    'Age 40t o 44': 'Age 40 to 44',
    'Age 45 t o49': 'Age 45 to 49',
    'Age 50t o 54': 'Age 50 to 54',
    'Age 55 to 59': 'Age 55 to 59',
    'Age 60 to6 4': 'Age 60 to 64',
    'Age 65 t o69': 'Age 65 to 69',
    'Age 70t o 74': 'Age 70 to 74',
    'Age 75t o 79': 'Age 75 to 79',
    'Age 80 ro older': 'Age 80 or older',
    'Age 80 or oldre': 'Age 80 or older',
    'Age 80 or loder': 'Age 80 or older',
    'Age 80o r older': 'Age 80 or older',
    'Age 80 oro lder': 'Age 80 or older',
    'Age 80 or odler': 'Age 80 or older',
    'Age 18 to 24': 'Age 18 to 24',
    'Age 25 t o29': 'Age 25 to 29',
    'Age 30 to 34': 'Age 30 to 34',
    'Age 35 to 39': 'Age 35 to 39',
    'Age 40 to 44': 'Age 40 to 44',
    'Age 45 to 49': 'Age 45 to 49',
    'Age 50 to 54': 'Age 50 to 54',
    'Age 55 to 59': 'Age 55 to 59',
    'Age 60 to 64': 'Age 60 to 64',
    'Age 65 to 69': 'Age 65 to 69',
    'Age 70 to 74': 'Age 70 to 74',
    'Age 75 to 79': 'Age 75 to 79',
    'Age 80 or older': 'Age 80 or older',
    'Age 45t o 49': 'Age 45 to 49',
    'Age 50 ot 54': 'Age 50 to 54',
    'Age 55t o 59': 'Age 55 to 59',
    'Age 60 to 46': 'Age 60 to 64',
    'Age 65 ot 69': 'Age 65 to 69',
    'Age 70t o 74': 'Age 70 to 74',
    'Age 75 to7 9': 'Age 75 to 79',
    'Age 80or older': 'Age 80 or older',
    'Age 30 to 43': 'Age 30 to 34',
    'Age 35 to 93': 'Age 35 to 39',
    'Age 45 to94': 'Age 45 to 49',
    'Age 50 to 54': 'Age 50 to 54',
    'Age 55 to95': 'Age 55 to 59',
    'Age 25 to92': 'Age 25 to 29',
    'Age 30 ot 34': 'Age 30 to 34',
    'Age 30t o 34': 'Age 30 to 34',
    'Age 35 ot 39': 'Age 35 to 39',
    'Age 45 ot 49': 'Age 45 to 49',
    'Age 55 to5 9': 'Age 55 to 59',
    'Age 30t o34': 'Age 30 to 34',
    'Age 35t o 39': 'Age 35 to 39',
    'Age 50 to5 4': 'Age 50 to 54',
    'Age 55 to 95': 'Age 55 to 59',
    'Age 65 to 96': 'Age 65 to 69',
    'Age 30 to34': 'Age 30 to 34',
    'Age 25 to 29': 'Age 25 to 29',
    'Age 35 to 39': 'Age 35 to 39',

    #for 'TetanusLast10Tdap' column
     'Yes, received Tdap': 'Yes, received Tdap',
    'Yes, received tetanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not recevie any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive anyt etanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive nay tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tteanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'No, did not receive any tetanus soht in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No ,did not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did notr eceive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, receievd Tdap': 'Yes, received Tdap',
    'Yes, receivedt etanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetanus shot in the past 10 yeras': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive any tetanus shoti n the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive any ettanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive any tetanus shot in the past 10 yaers': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, reecived Tdap': 'Yes, received Tdap',
    'Yes, reecived Tdap': 'Yes, received Tdap',
    'Yes, erceived tetanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'No, did not receive any tetanus shot i nthe past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes ,received Tdap': 'Yes, received Tdap',
    'eYs, received Tdap': 'Yes, received Tdap',
    'Yes,r eceived Tdap': 'Yes, received Tdap',
    'No, did not receive any tetanus shot in the apst 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot but not usre what type': 'Yes, received tetanus shot but not sure what type',
    'No, ddi not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not reecive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'oN, did not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received Tadp': 'Yes, received Tdap',
    'Yes, received tetnaus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetauns shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received Tdpa': 'Yes, received Tdap',
    'Yes, received tetanus shot butn ot sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes ,received tetanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Ye,s received tetanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'No, di dnot receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No,d id not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive any tetanus shot in hte past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'No, did not receive any tetanus shto in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot but not sure whta type': 'Yes, received tetanus shot but not sure what type',
    'No, did not receive any tetnaus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, receivde Tdap': 'Yes, received Tdap',
    'No, did not receive any tetanus shot in the past 10 eyars': 'No, did not receive any tetanus shot in the past 10 years',
    'Yse, received Tdap': 'Yes, received Tdap',
    'Yes, rceeived Tdap': 'Yes, received Tdap',
    'No, did not receive any tetanus shot in thep ast 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot, butn ot Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetauns shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'No, did not receive any tetanus shot in teh past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot but not suer what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus shot ubt not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus shot, but not Tdpa': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot but not sure wha ttype': 'Yes, received tetanus shot but not sure what type',
    'Yes, received teatnus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetanus shot in the past 01 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot bu tnot sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetansu shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot but not srue what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus soht, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetanus shot in the past 10 yeasr': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot but no tsure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus sho tbut not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus shot, but no tTdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot but not sure wha ttype': 'Yes, received tetanus shot but not sure what type',
    'Yes, received teatnus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetansu shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus shot, but not Tadp': 'Yes, received tetanus shot but not Tdap',
    'eYs, received tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, recevied tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot, but notT dap': 'Yes, received tetanus shot but not Tdap',
    'Yes, receievd tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot but not sure whatt ype': 'Yes, received tetanus shot but not sure what type',
    'Yes, rceeived tetanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'No, did not receive any tteanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot, ubt not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did ont receive any tetanus shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetnaus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, rceeived tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanu sshot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot, but nto Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shto but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus hsot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, recieved tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot but not sure hwat type': 'Yes, received tetanus shot but not sure what type',
    'Yes, receivde tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'No, did not receive any tetansu shot in the past 10 years': 'No, did not receive any tetanus shot in the past 10 years',
    'Yes, received tetanus shot,b ut not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tteanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, receivde tetanus shot but not sure what type': 'Yes, received tetanus shot but not sure what type',
    'Yes, received tetanus shot ,but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes ,received tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, reecived tetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received tetanus shot, but not dTap': 'Yes, received tetanus shot but not Tdap',
    'Yes, receive dtetanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    'Yes, received ettanus shot, but not Tdap': 'Yes, received tetanus shot but not Tdap',
    # Add more typos and corrections here
}

# List of columns where replacements should be applied
columns_to_replace = ['PhysicalActivities','HadHeartAttack','HadAngina','HadStroke','HadAsthma','HadSkinCancer','HadCOPD','HadDepressiveDisorder','HadKidneyDisease'
                      ,'HadArthritis','HadDiabetes','DeafOrHardOfHearing','BlindOrVisionDifficulty','DifficultyConcentrating','DifficultyWalking','DifficultyDressingBathing'
                      , 'GeneralHealth', 'LastCheckupTime', 'DifficultyErrands','ChestScan','AlcoholDrinkers','HIVTesting','PneumoVaxEver','TetanusLast10Tdap','HighRiskLastYear'
                      ,'CovidPos','FluVaxLast12','Sex','LastCheckupTime','RemovedTeeth','State','SmokerStatus','ECigaretteUsage','RaceEthnicityCategory','AgeCategory'
                     ,'TetanusLast10Tdap']

# Apply the replacements across the specified columns
data = data.replace(replacements, subset=columns_to_replace)
